In [2]:
import langchain 
langchain.__version__

'0.3.20'

In [3]:
from langchain.agents import initialize_agent, Tool
# from langchain_community.llms import Ollama
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
import os


In [4]:
load_dotenv()

API_KEY = os.getenv("GOOGLE_AI_API")

# Search Web

In [5]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

search_tool = Tool(
    name = "DuckDuckGo Search",
    func= DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches using DuckDuckGo"
)

llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=API_KEY)

tool = [search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("What are the latest top running movies in 2025")

/tmp/ipykernel_8500/1839646698.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)
/tmp/ipykernel_8500/1839646698.py:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What are the latest top runn



> Entering new AgentExecutor chain...
I need to find the top running movies in 2025. Since it's impossible to know what will be top running in the future, I will search for predictions or lists of anticipated movies for 2025.

Action: DuckDuckGo Search
Action Input: "most anticipated movies 2025"
Observation: Find out what movies are coming in 2025, from superhero blockbusters to musical biopics, horror films to action thrillers. See the release dates, directors, stars, and plot summaries of the most noteworthy titles of the year. Find out what's coming to the big screen in 2025, from superhero sequels to musical adaptations to horror remakes. Rolling Stone's David Fear previews the most exciting films of the year, featuring stars like Tom Cruise, Cate Blanchett, Ariana Grande and more. Find out which movies ticket-buyers are looking forward to in 2025, from Captain America to Jurassic World. See the rankings, genres, performances and more in this article by Fandango staff. Here are 

# Wikipedia Search

In [6]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wiki_search_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search Wikipedia for information on any topic"
)

llm = GoogleGenerativeAI(model="gemini-2.0-flash",google_api_key=API_KEY)


tool = [wiki_search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("Who is Elon Musk? Tell me the latest news of him")

print(response)



> Entering new AgentExecutor chain...
I need to find out who Elon Musk is and then search for recent news about him. I'll start by searching Wikipedia for a general overview of Elon Musk.
Action: Wikipedia Search
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman known for his key roles in Tesla, Inc., SpaceX, and Twitter (which he rebranded as X). Since 2025, he has been a senior advisor to United States president Donald Trump and the de facto head of the Department of Government Efficiency (DOGE). Musk is the wealthiest person in the world; as of March 2025, Forbes estimates his net worth to be US$343 billion. 
Born to a prominent family in Pretoria, South Africa, Musk emigrated to Canada in 1989 and acquired its citizenship though his mother. He moved to the U.S. and graduated from the University of Pennsylvania before moving to California to pursue business ventures. In 1995, Musk co-founded the software 

# SQLite database

In [7]:
import sqlite3

connection = sqlite3.connect("example_new.db")
cursor = connection.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
        id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        stock_quantity INTEGER NOT NULL
)
""")

cursor.executemany("""
INSERT INTO inventory(product_name, stock_quantity) VALUES(?,?)
""", [
    ("Product A", 150),
    ("Product B", 250),
    ("Product C", 350),
    ("Product D", 450),
    ("Product E", 550),
])

connection.commit()
connection.close()

print("Database created and populated with sample data")

Database created and populated with sample data


In [8]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

In [9]:
db = SQLDatabase.from_uri("sqlite:///example_new.db")

llm = GoogleGenerativeAI(model="gemini-2.0-flash",google_api_key=API_KEY)

toolkit = SQLDatabaseToolkit(db=db,llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

query1="How many units of Product A are in stock ?"
response1 = agent.run(query1)
print("Response:",response1)



> Entering new AgentExecutor chain...
I need to find the number of units of Product A in stock. I should start by listing the tables in the database to see what tables are available.
Action: sql_db_list_tables
Action Input:
Observation: inventory
Thought:I should examine the schema of the `inventory` table to understand its structure and find the relevant columns for product name and stock quantity.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Product A	150
2	Product B	250
3	Product C	350
*/
Thought:I need to query the `inventory` table to find the `stock_quantity` for `product_name` equal to 'Product A'.
Action: sql_db_query_checker
Action Input: SELECT stock_quantity FROM inventory WHERE product_name = 'Product A'
Observation: SELECT stock_quantity FROM inventory WHERE product_nam

In [10]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun

In [ ]:
arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool search academic papers on arXiv. Provide topics, authors, or keywords."
)

llm = GoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)

tools = [arxiv_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

query = "Search for research papers on Object Detection in 2024"
response = agent.run(query)
print("Response from Agent:")
print(response)